# Assignment-3 COMP-5630 Jacob Murrah
## README
This notebook implements natural language processing tasks focusing on word embeddings, n-gram classification, and sequence-to-sequence models for text summarization. It explores different embedding techniques, evaluates n-gram features for classification, and implements RNN-based models for summarizing legislative text.

## Dependencies
- **Python 3.x**
- **matplotlib**
- **seaborn**
- **nltk**
- **numpy==1.26.4**
- **torch==2.2.2**
- **transformers==4.50.3**
- **datasets==2.18.0**
- **scipy==1.12.0**
- **pandas==2.1.4**
- **scikit-learn==1.4.0**
- **gensim==4.3.2**
- **evaluate==0.4.1**

## Instructions
1. Run the cell with the skip cell magic function defined. Marked with the # SKIP MAGIC FUNCTION comment.
2. Run the cell beginning with %%skip and marked with the # DOWNLOAD PACKAGES CELL comment.
3. After downloads are complete the runtime will be disconnected and you will see an associated message.
4. Reconnect the runtime and click on \"Runtime\" > \"Run all\" to execute the entire notebook sequentially.
6. The notebook is organized into several sections. Ensure that all cells run without errors.

In [ ]:
SKIP_INSTALL = True

In [ ]:
# SKIP MAGIC FUNCTION
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
  if globals().get('SKIP_INSTALL', False):
    print("Skipping installation cell.")
  else:
    get_ipython().run_cell(cell)

In [ ]:
# SKIP_INSTALL = False

In [ ]:
%%skip
# DOWNLOAD PACKAGES CELL
import contextlib
import io
import time
import sys
import subprocess
import re
import os

def capture_output():
  return contextlib.redirect_stdout(io.StringIO())

print("Downloads take ~5 minutes but are required for version compatiability!")
time.sleep(1)

with capture_output():
  package_versions = {
    "numpy": "1.26.4",
    "torch": "2.2.2",
    "transformers": "4.50.3",
    "datasets": "2.18.0",
    "scipy": "1.12.0",
    "pandas": "2.1.4",
    "scikit-learn": "1.4.0",
    "gensim": "4.3.2",
    "evaluate": "0.4.1"
  }

  # get cuda version
  cuda_version_str = "N/A"
  cuda_major_minor = None
  nvcc_output = subprocess.check_output(['nvcc', '--version']).decode('utf-8')
  match_obj = re.search(r'release (\d+\.\d+)', nvcc_output)
  if match_obj:
    cuda_version_str = match_obj.group(1)
    cuda_major_minor = "".join(cuda_version_str.split('.'))

  # pytorch installation
  pytorch_version = package_versions["torch"]
  pytorch_cuda_suffix = "cu118"
  if cuda_major_minor and int(cuda_major_minor) >= 121:
    pytorch_cuda_suffix = "cu121"

  torchvision_version = "0.17.2"
  torchaudio_version = "2.2.2"
  pytorch_index_url = f"https://download.pytorch.org/whl/{pytorch_cuda_suffix}"

  pytorch_install_command = (
    f"pip3 install "
    f"torch=={pytorch_version}+{pytorch_cuda_suffix} "
    f"torchvision=={torchvision_version}+{pytorch_cuda_suffix} "
    f"torchaudio=={torchaudio_version}+{pytorch_cuda_suffix} "
    f"--index-url {pytorch_index_url} --upgrade --no-cache-dir"
  )
  get_ipython().system(pytorch_install_command)

  # install specified packages
  get_ipython().system('pip3 uninstall transformers -y')
  packages_to_install = [
    f"{pkg}=={ver}" for pkg, ver in package_versions.items() if pkg != "torch"
  ]
  other_packages_install_command = (
    "pip3 install --upgrade --no-cache-dir " + " ".join(packages_to_install)
  )

  get_ipython().system(other_packages_install_command)
  get_ipython().system(
    'pip install "huggingface_hub[hf_xet]" --upgrade --no-cache-dir'
  )

# restart environment
print("Package installation completed successfully!")
os._exit(0)

In [ ]:
# basic imports
import gensim
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

import importlib.metadata
import sys
version = importlib.metadata.version("torch")
print(version)

# Part 1. Word Embeddings and N-gram

In [ ]:
## imports for word embedding
import nltk
from nltk.corpus import twitter_samples, stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import gensim.downloader as api
from gensim.models import FastText, Word2Vec
from gensim.test.utils import datapath

from sklearn.metrics.pairwise import cosine_similarity

import re
import string

## Part 1. (1a-1b) You will examine two-word embeddings. You are given the following words:

Dog, Bark, Tree, Bank, River, Money.

Do the following:
- Use Glove-twitter-50D word2vec and compute nxn matrices using cosine similarities for the given words.
- Use FastText and compute nxn matrices using cosine similarities for the given words.

In [ ]:
# download glove-twitter-50 model
nltk.download("twitter_samples")
nltk.download('stopwords')
if "glove" not in globals():
  glove = api.load("glove-twitter-50")

print("downloads completed!")

In [ ]:
# data preprocessing
# https://www.kaggle.com/code/piyushagni5/sentiment-analysis-on-twitter-dataset-nlp
def process_tweet(tweet):
  stemmer = PorterStemmer()
  stopwords_english = stopwords.words('english')

  tweet = re.sub(r'\$\w*', '', tweet)
  tweet = re.sub(r'^RT[\s]+', '', tweet)
  tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
  tweet = re.sub(r'#', '', tweet)

  tokenizer = TweetTokenizer(
    preserve_case=False, strip_handles=True, reduce_len=True
  )
  tweet_tokens = tokenizer.tokenize(tweet)

  tweets_clean = []
  for word in tweet_tokens:
    if (word not in stopwords_english and word not in string.punctuation):
      stem_word = stemmer.stem(word)
      tweets_clean.append(stem_word)

  return tweets_clean

negative_tweets = twitter_samples.strings("negative_tweets.json")
positive_tweets = twitter_samples.strings("positive_tweets.json")
common_texts = [
  process_tweet(tweet)
  for tweet_list in [negative_tweets, positive_tweets]
  for tweet in tweet_list
]
print(f"Number of tweets: {len(common_texts)}")

In [ ]:
# train the Word2Vec and FastText models
# NOTE: our GloVe w2v model is already loaded
if "w2v" not in globals():
  w2v = Word2Vec(sentences=common_texts, vector_size=50, window=5, min_count=1)
if "ft" not in globals():
  ft = FastText(
    vector_size=50, window=5, min_count=1, sentences=common_texts, epochs=10
  )
print("done training models!")

In [ ]:
# create similarity matrices of the words for each model
def create_similarity_matrix(model, words, glove_model=False):
  if glove_model:
    common_text_words = set([word for text in common_texts for word in text])
    glove_words = [
      word.lower() for word in glove.key_to_index
      if word.lower() in common_text_words
    ]
    words_in_model = [w for w in words if w in glove_words]
  else:
    words_in_model = [w for w in words if w in model.wv]

  word_vectors = np.array(
    [
      model.get_vector(word) if glove_model else model.wv[word]
      for word in words_in_model
    ]
  )
  similarity_matrix = cosine_similarity(word_vectors)
  return similarity_matrix, words_in_model

words = ["dog", "bark", "tree", "bank", "river", "money"]
glove_matrix, words_in_glove = create_similarity_matrix(
  glove, words, glove_model=True
)
w2v_matrix, words_in_w2v = create_similarity_matrix(w2v, words)
ft_matrix, words_in_ft = create_similarity_matrix(ft, words)

In [ ]:
# plot the similarity matrices for each model
def plot_similarity_matrices(matrices, words_lists, titles):
  n = len(matrices)
  fig, axes = plt.subplots(1, n, figsize=(n * 6, 5))
  for i in range(n):
    sns.heatmap(
      matrices[i],
      annot=True,
      fmt=".2f",
      cmap="YlGnBu",
      xticklabels=words_lists[i],
      yticklabels=words_lists[i],
      ax=axes[i]
    )
    axes[i].set_title(titles[i], fontsize=14)

  plt.tight_layout()
  plt.show()

matrices = [glove_matrix, w2v_matrix, ft_matrix]
words_lists = [words_in_glove, words_in_w2v, words_in_ft]
titles = ["GloVe Similarity", "Word2Vec Similarity", "FastText Similarity"]
plot_similarity_matrices(matrices, words_lists, titles)

## Part 1. (1c) Which embedding captures better semantics? Justify your answer.<br>
**Answer:** Based on the similarity scores between the 6 words, the GloVe model is better at capturing the semantics, i.e the meaning, of the words. The FastText model is a close second. Since there is not a universal standard similarity score for these specific word pairs I am using my own understanding of the english language to evaluate.

Words with multiple meanings:
- Bark: Dog barking or tree bark
- Bank: River bank or a bank that manages money

Related pairs:
- Dog-Bark
- Tree-Bark
- Money-Bank
- River-Bank

Model Evaluation:
- **GloVe:** The similarity matrix shows clear differentiation between semantically distinct words. For example, the similarity between “bark” and “bank” is extremely low and the similarity between "money" and "bank" is relatively high.
- **Word2Vec:** Shows high similarity scores across most of the pairs and does not differentiate unrelated words. For example, the words "dog" and "tree" are unrelated but have an extremely high similarity score.
- **FastText:** Is able to capture semantic of words but at a more moderate extent than GloVe. This moderate scoring does not differentiate as clearly between related and unrelated pairs as GloVe does.

Based only on their 6-word similarity matrices, the GloVe model provides more realistic similarity scores and thus captures the semantic similarities between words better than the other models.

In [ ]:
benchmark = "simlex999.txt"
result_glove = glove.evaluate_word_pairs(datapath(benchmark))
result_w2v = w2v.wv.evaluate_word_pairs(datapath(benchmark))
result_ft = ft.wv.evaluate_word_pairs(datapath(benchmark))

print(f"GloVe Spearman Measurements:")
print(f" - Coefficient: {result_glove[1][0]:.4f}")
print(f" - P-Value: {result_glove[1][1]:.4f}")

print(f"Word2Vec Spearman Measurements:")
print(f" - Coefficient: {result_w2v[1][0]:.4f}")
print(f" - P-Value: {result_w2v[1][1]:.4f}")

print(f"FastText Spearman Measurements:")
print(f" - Coefficient: {result_ft[1][0]:.4f}")
print(f" - P-Value: {result_ft[1][1]:.4f}")

## Part 1. (1c) Continued analysis...<br>
**Answer:** I wanted to see which model performed better on a larger set of words so I measured the Spearman Coefficients of each model on the 'simlex999' dataset. In our context, the Spearman Coefficient judges how well our model's predictions match a human's analysis of word pairs. A higher coefficient shows a better match. On the 'simlex999' dataset, the GloVe model achieves the highest Spearman coefficient and is the only model with a statistically significant p-value, indicating that it does the best at capturing overall semantic relationships across a broader range of word pairs.

## Part 1. (2a-2c) N-grams and Classification

Do the following:
- Split the data 70% training and 30% testing.
- Extract n-grams for n in [1, 4]. unigram, bigram, trigram, 4-grams.
- Build a logistic regression model using n-gram features.

In [ ]:
import random
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams

In [ ]:
# split the data into 70% training and 30% testing
seed = 2025
random.seed(seed)
labels = np.array([0] * len(negative_tweets) + [1] * len(positive_tweets))
train_texts, test_texts, train_labels, test_labels = train_test_split(
  common_texts, labels, test_size=0.3, random_state=seed
)

In [ ]:
# extract n-grams for n in [1, 4]. unigram, bigram, trigram, 4grams.
def get_text_ngrams(text, n_range):
  text_ngrams = []
  for i in n_range:
    text_string = ["_".join(gram) for gram in list(ngrams(text, i))]
    text_ngrams.extend(text_string)

  return " ".join(text_ngrams)

train_features = [get_text_ngrams(text, range(1, 5)) for text in train_texts]
test_features = [get_text_ngrams(text, range(1, 5)) for text in test_texts]

In [ ]:
# logistic regression model
def train_and_predict_model(
  train_features, test_features, train_labels, test_labels
):
  vectorizer = CountVectorizer()
  X_train = vectorizer.fit_transform(train_features)
  X_test = vectorizer.transform(test_features)

  model = LogisticRegression(max_iter=1000)
  model.fit(X_train, train_labels)

  y_pred = model.predict(X_test)
  accuracy = accuracy_score(test_labels, y_pred)
  report = classification_report(test_labels, y_pred)

  return accuracy, report

accuracy, report = train_and_predict_model(
  train_features, test_features, train_labels, test_labels
)
print(f'Accuracy: {accuracy:.4f}')
print(report)

## Part 1. (2c) Evaluate your logistic regression model's performance.<br>
**Answer:** My logistic regression model has an accuracy of ~73.5% with similar performance for predicting positive and negative sentiment in tweets. My model has better precision (fewer false positives) for positive sentiment tweets and better recall (fewer false negatives) for negative sentiment tweets. These results show that even a simple logistic regression model can predict sentiment in tweets with decent accuracy when trained with n-grams.

## Part 1. (2d) Plotting model accuracy vs n

In [ ]:
# compare cumulative vs stand alone ngram accuracies
def get_accuracy_scores(train_texts, test_tests, n_range, cumulative):
  accuracy_scores = []
  for i in n_range:
    r = range(i+1) if cumulative else range(i, i+1)
    train_features = [get_text_ngrams(text, r) for text in train_texts]
    test_features = [get_text_ngrams(text, r) for text in test_texts]

    accuracy, report = train_and_predict_model(
      train_features, test_features, train_labels, test_labels
    )
    print(
      f"{'CUMULATIVE' if cumulative else 'SINGLE'}, n={i}, acc={accuracy:.4f}"
    )
    accuracy_scores.append(accuracy)

  return accuracy_scores

n_range = range(1, 10)
cumulative_accs = get_accuracy_scores(train_texts, test_texts, n_range, True)
single_accs = get_accuracy_scores(train_texts, test_texts, n_range, False)

In [ ]:
# plot the results
def create_accuracy_plot(ax, n_range, accuracies, title, cumulative=False):
  marker = 's' if cumulative else 'o'
  color = '#ff7f0e' if cumulative else '#1f77b4'
  ax.plot(
    n_range,
    accuracies,
    marker=marker,
    linestyle='-',
    color=color,
    linewidth=2,
    markersize=8
  )
  ax.set_xlabel('Max N-gram Size (n)' if cumulative else 'N-gram Size (n)')
  ax.set_ylabel('Accuracy')
  ax.set_title(title, fontsize=14)
  ax.grid(True, linestyle='--', alpha=0.7)
  ax.set_xticks(n_range)

  for i, acc in enumerate(accuracies):
    ax.text(n_range[i], acc + 0.001, f'{acc:.4f}', ha='center', fontsize=9)

def plot_ngram_comparison(n_range, single_accs, cumulative_accs):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
  create_accuracy_plot(
    ax1, n_range, single_accs, 'Individual N-gram Size'
  )
  create_accuracy_plot(
    ax2, n_range, cumulative_accs, 'Cumulative N-gram Size', True
  )

  plt.tight_layout()
  plt.show()

plot_ngram_comparison(n_range, single_accs, cumulative_accs)

## Part 1. (2d) How does the value of n in n-gram affect the model's performance?
Explain your answer. You can draw a plot with n-gram and the model's performance.<br>
**Answer:** The size of n and wether or not the n-grams are cumulative both have an impact on the model's accuracy.

In terms of individual contributions to the model's accuracy, the unigrams have the most impact and provide the highest accuracy. This makes sense because individual words provide the majority of the sentiment in sentences. For cumulative n-grams, a maximum size of 3 provides the highest accuracy. After this point increasing the size of n leads to decreasing accuracy. Including up to trigrams provide the best accuracy because sentiment is often conveyed in 1-3 words in the english language. For example, two words are a requirement to convey the sentiment when saying something is 'not good'.

Overall unigrams, bigrams, and trigrams make up most of the contributions to the model's accuracy. Any additional increases to n lead to diminishing accuracy because a majority of the sentiment is captured in just 1-3 word n-grams.

# Part 2. RNN and Machine Translation
You will be training a Seq2seq model using RNN. Your input will be a text and the output will be a summary of the text.

In [ ]:
# imports for RNN
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import nltk
from nltk.translate.bleu_score import (
  corpus_bleu, sentence_bleu, SmoothingFunction
)
from transformers import AutoTokenizer
from transformers.optimization import Adafactor
from datasets import load_dataset
import random
import time
import numpy as np
from tqdm.auto import tqdm

seed = 2025
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Part 2. (1) Load the California State bill subset of the BillSum dataset from HuggingFace.
Do the following:
- Load the test split as your entire dataset for this task.
- Split the dataset into a train and test set with the train test split method.

In [ ]:
# download and load the data
# https://huggingface.co/docs/transformers/main/en/tasks/summarization
nltk.download('punkt')
nltk.download('punkt_tab')
billsum = load_dataset("billsum", split="ca_test").train_test_split(
  test_size=0.2, seed=seed
)
billsum_train, billsum_test = billsum["train"], billsum["test"]

## Part 2. (2) Use the number of neurons, dropout, and your selection of RNN architecture.
NOTE: Report BLEU as the model’s performance.

In [ ]:
# custom dataset for our seq2seq model
class Seq2SeqDataset(Dataset):
  def __init__(self, data, combination):
    data = data.map(self.preprocess, combination)
    self.input_ids = data["input_ids"]
    self.labels = data["labels"]

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, index):
    return {
      "input_ids": torch.tensor(self.input_ids[index], dtype=torch.long),
      "labels": torch.tensor(self.labels[index], dtype=torch.long)
    }

  def preprocess(
    self, example, input_max_length=1024, summary_max_length=128
  ):
    tokenized_input = tokenizer.encode_plus(
      "summarize the legal document: " + example["text"],
      max_length=input_max_length,
      truncation=True,
      padding="max_length"
    )
    tokenized_summary = tokenizer.encode_plus(
      example["summary"],
      max_length=summary_max_length,
      truncation=True,
      padding="max_length"
    )
    return {
      "input_ids": tokenized_input["input_ids"],
      "labels": tokenized_summary["input_ids"]
    }

In [ ]:
# seq2seq model w/ LSTM encoder and decoder
class Seq2SeqModel(nn.Module):
  def __init__(self, vocab_size, embed_size, hidden_size, dropout=0.1):
    super(Seq2SeqModel, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embed_size)
    self.encoder = nn.LSTM(embed_size, hidden_size, batch_first=True)
    self.decoder = nn.LSTM(embed_size, hidden_size, batch_first=True)
    self.output_layer = nn.Linear(hidden_size, vocab_size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, source, target):
    source_embedding = self.dropout(self.embedding(source))
    _, (h, c) = self.encoder(source_embedding)

    target_embedding = self.dropout(self.embedding(target))
    decoder_output, _ = self.decoder(target_embedding, (h, c))
    logits = self.output_layer(decoder_output)
    return logits

In [ ]:
# train model with specified max tokenization lengths
def train_model(model, train_loader, optimizer, criterion, device, epochs=3):
  model.train()
  for i in range(epochs):
    epoch_loss = 0.0
    for batch in train_loader:
      source = batch["input_ids"].to(device)
      labels = batch["labels"].to(device)

      decoder_input = labels[:,:-1]
      decoder_target = labels[:,1:]

      optimizer.zero_grad()
      logits = model(source, decoder_input)
      logits = logits.reshape(-1, logits.size(-1))
      decoder_target = decoder_target.reshape(-1)

      loss = criterion(logits, decoder_target)
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()

    print(f"Epoch {i+1}/{epochs} - Loss: {epoch_loss / len(train_loader):.4f}")

## Part 2. (3) Vary the input seq length by truncating the main text and the summary text.
We will determine how these lengths impact the model's performance:
- Main Text Lengths: 1024, 2048
- Summary Text Lengths: 128, 256

In [ ]:
# calculate the corpus-level BLEU-4 score
def calculate_bleu(model, test_loader, tokenizer, device):
  model.eval()
  predictions = []
  references = []

  smoothing = SmoothingFunction().method1
  with torch.no_grad():
    for batch in test_loader:
      source = batch["input_ids"].to(device)
      labels = batch["labels"].to(device)

      decoder_input = labels[:,:-1]

      outputs = model(source, decoder_input)
      pred_ids = torch.argmax(outputs, dim=-1)

      decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
      decoded_refs = tokenizer.batch_decode(labels, skip_special_tokens=True)

      predictions.extend(decoded_preds)
      references.extend([[reference] for reference in decoded_refs])

  bleu_score = corpus_bleu(
    references,
    predictions,
    weights=(0.25, 0.25, 0.25, 0.25),
    smoothing_function=smoothing
  )
  return bleu_score

In [ ]:
# pad a batch of variable length tokens to be uniform length
def collate_fn(batch):
  input_ids = []
  labels = []
  for item in batch:
    input_ids.append(item["input_ids"].clone().detach())
    labels.append(item["labels"].clone().detach())

  padded_input_ids = pad_sequence(
    input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
  )
  padded_labels = pad_sequence(
    labels, batch_first=True, padding_value=tokenizer.pad_token_id
  )
  return {"input_ids": padded_input_ids, "labels": padded_labels}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
seq_length_combinations = [
  {'input_max_length': 1024, 'summary_max_length': 128},
  {'input_max_length': 1024, 'summary_max_length': 256},
  {'input_max_length': 2048, 'summary_max_length': 128},
  {'input_max_length': 2048, 'summary_max_length': 256},
]

for seq_length_combination in seq_length_combinations:
  train_dataset = Seq2SeqDataset(billsum_train, seq_length_combination)
  test_dataset = Seq2SeqDataset(billsum_test, seq_length_combination)
  train_loader = DataLoader(
    train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn
  )
  test_loader  = DataLoader(
    test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn
  )

  vocab_size = tokenizer.vocab_size
  embed_size = 256
  hidden_size = 512

  model = Seq2SeqModel(
    vocab_size, embed_size, hidden_size, dropout=0.1
  ).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

  train_model(model, train_loader, optimizer, criterion, device, epochs=3)
  bleu_score = calculatue_bleu(model, test_loader, tokenizer, device)
  print(seq_length_combination)
  print(f"BLEU Score: {bleu_score:.4f}")

## Part 2. (3) How does the sequence length impact the model’s performance?
**Answer:** The largest impact on the BLEU score was due to increasing the summary length from 128 to 256.